In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

cutoff_point = 600
application_types_to_replace = application_count[application_count < cutoff_point].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [5]:
classification_count = application_df['CLASSIFICATION'].value_counts()

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_point = 1000
classifications_to_replace = classification_count[classification_count < cutoff_point].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Convert categorical data to numeric with `pd.get_dummies`
numeric_application_df = pd.get_dummies(application_df)


In [10]:
# Split our preprocessed data into our features and target arrays
y = numeric_application_df['IS_SUCCESSFUL']
x = numeric_application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
### TRAINING MODEL

In [13]:
# setting up the model using keras
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  18
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 18)                774       
                                                                 
 dense_1 (Dense)             (None, 11)                209       
                                                                 
 dense_2 (Dense)             (None, 9)                 108       
                                                                 
 dense_3 (Dense)             (None, 1)                 10        
                                                                 
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6044 - accuracy: 0.6985
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5737 - accuracy: 0.7270
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5670 - accuracy: 0.7283
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5632 - accuracy: 0.7287
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5610 - accuracy: 0.7287
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.7288
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5571 - accuracy: 0.7288
Epoch 8/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7295
Epoch 9/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5546 - accuracy: 0.7292
Epoch 10/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5537 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5411 - accuracy: 0.7361
Epoch 82/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7371
Epoch 83/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7375
Epoch 84/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5405 - accuracy: 0.7374
Epoch 85/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5401 - accuracy: 0.7378
Epoch 86/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5404 - accuracy: 0.7358
Epoch 87/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5400 - accuracy: 0.7368
Epoch 88/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5401 - accuracy: 0.7371
Epoch 89/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7373
Epoch 90/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5401 - accuracy: 

804/804 [==============================] - 3s 4ms/step - loss: 0.5377 - accuracy: 0.7383
Epoch 161/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5373 - accuracy: 0.7371
Epoch 162/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5379 - accuracy: 0.7371
Epoch 163/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5377 - accuracy: 0.7377
Epoch 164/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5377 - accuracy: 0.7382
Epoch 165/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5377 - accuracy: 0.7379
Epoch 166/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5375 - accuracy: 0.7384
Epoch 167/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5377 - accuracy: 0.7387
Epoch 168/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5381 - accuracy: 0.7370
Epoch 169/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5374 - a

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5591 - accuracy: 0.7192 - 499ms/epoch - 2ms/step
Loss: 0.5591253638267517, Accuracy: 0.7191836833953857


In [18]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_v1.h5')

In [20]:
### ATTEMPT NUMBER 2
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 40)                1720      
                                                                 
 dense_9 (Dense)             (None, 40)                1640      
                                                                 
 dense_10 (Dense)            (None, 40)                1640      
                                                                 
 dense_11 (Dense)            (None, 1)                 41        
                                                                 
Total params: 5,041
Trainable params: 5,041
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5721 - accuracy: 0.7188
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 -

804/804 [==============================] - 2s 3ms/step - loss: 0.5348 - accuracy: 0.7394
Epoch 73/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5347 - accuracy: 0.7395
Epoch 74/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7393
Epoch 75/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5347 - accuracy: 0.7399
Epoch 76/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7406
Epoch 77/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5343 - accuracy: 0.7397
Epoch 78/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5341 - accuracy: 0.7400
Epoch 79/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5344 - accuracy: 0.7400
Epoch 80/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5346 - accuracy: 0.7393
Epoch 81/200
804/804 [==============================] - 5s 6ms/step - loss: 0.5343 - accuracy: 

804/804 [==============================] - 2s 3ms/step - loss: 0.5320 - accuracy: 0.7410
Epoch 152/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7411
Epoch 153/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7399
Epoch 154/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7416
Epoch 155/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5311 - accuracy: 0.7413
Epoch 156/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7408
Epoch 157/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5313 - accuracy: 0.7404
Epoch 158/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7406
Epoch 159/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5314 - accuracy: 0.7418
Epoch 160/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - a

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5831 - accuracy: 0.7272 - 477ms/epoch - 2ms/step
Loss: 0.5831170082092285, Accuracy: 0.7272303104400635


In [22]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_v2.h5')

In [25]:
#### ATTEMPT # 3
# setting up the model using keras
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  18
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 18)                774       
                                                                 
 dense_21 (Dense)            (None, 11)                209       
                                                                 
 dense_22 (Dense)            (None, 9)                 108       
                                                                 
 dense_23 (Dense)            (None, 1)                 10        
                                                                 
 dense_24 (Dense)            (None, 1)                 2         
                                                                 
Total params: 1,103
Trainable params: 1,103
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
804/804 [==============================]

804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7366
Epoch 71/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7349
Epoch 72/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7369
Epoch 73/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7357
Epoch 74/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5457 - accuracy: 0.7359
Epoch 75/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7356
Epoch 76/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7358
Epoch 77/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7350
Epoch 78/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5450 - accuracy: 0.7366
Epoch 79/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 

Epoch 150/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5394 - accuracy: 0.7371
Epoch 151/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5394 - accuracy: 0.7365
Epoch 152/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5395 - accuracy: 0.7374
Epoch 153/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5392 - accuracy: 0.7383
Epoch 154/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5395 - accuracy: 0.7383
Epoch 155/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5391 - accuracy: 0.7389
Epoch 156/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5391 - accuracy: 0.7375
Epoch 157/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5390 - accuracy: 0.7382
Epoch 158/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5393 - accuracy: 0.7383
Epoch 159/200
804/804 [==============================] - 2s 2ms/step - lo

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_v3.h5')

268/268 - 0s - loss: 0.5585 - accuracy: 0.7270 - 475ms/epoch - 2ms/step
Loss: 0.5585041642189026, Accuracy: 0.7269970774650574
